# Class 8 Notebook – Bias and Ethics in AI/ML Basics

This notebook introduces **bias and ethics** in AI and machine learning.

As ML systems are deployed in hiring, lending, healthcare, and justice, understanding bias and fairness becomes essential:
- **Data bias** – Training data reflects historical inequities
- **Algorithmic bias** – Models can amplify or introduce new bias
- **Fairness** – Different definitions (e.g., demographic parity, equalized odds) and trade-offs

**Objective**: Set up the environment and explore core concepts for thinking critically about bias and responsible AI.

**Key ideas**:
- Bias can enter at data collection, labeling, feature selection, and model design
- Fairness metrics can conflict; there is no single "correct" definition
- Responsible AI includes transparency, interpretability, and ongoing monitoring

Run the first code cell to confirm your environment works.

## Run in the browser (no local setup)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adzuci/ai-fundamentals/blob/class-8-bias-and-ethics/class-8-bias-and-ethics/01_class_8_bias_and_ethics_basics.ipynb)

> Tip: This notebook assumes you're comfortable with basic Python, pandas, and scikit-learn from earlier classes.

## STEP 1: Environment check and imports

Verify that NumPy, pandas, and scikit-learn are available for building simple classifiers and analyzing predictions.

In [ ]:
# Concept: Environment sanity check for bias/ethics notebook
import platform
import numpy as np
import pandas as pd
import sklearn

print(f"Python: {platform.python_version()}")
print(f"OS: {platform.system()} {platform.release()}")
print(f"NumPy: {np.__version__}")
print(f"pandas: {pd.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print("All libraries imported successfully!")

## What is bias in ML?

**Bias** in machine learning refers to systematic errors or unfairness in model behavior—often toward or against certain groups.

- **Data bias**: Training data underrepresents groups, reflects historical discrimination, or has labeling errors that correlate with protected attributes.
- **Algorithmic bias**: The model itself (e.g., regularization, threshold choices) produces different error rates or outcomes across groups.
- **Feedback loops**: Deployed models influence future data (e.g., predictive policing), reinforcing existing bias.

In later cells, we'll add examples and simple fairness checks.

## STEP 2: Create a toy dataset for bias exploration

We create a simple dataset with **experience**, **test_score**, and **gender**—typical features in hiring or performance contexts. We'll use it to explore how models behave across groups and discuss fairness.

In [ ]:
# Concept: Create dataset for bias exploration
# import Libraries
import numpy as np
import pandas as pd

np.random.seed(42)
# Create Dataset
mydata = pd.DataFrame({
    "exp": np.random.randint(0, 10, 100),
    "test_score": np.random.randint(50, 100, 100),
    "gender": np.random.choice(["Male", "Female"], 100)
})